In [92]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [93]:
import numpy as np
import pandas as pd

In [94]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [95]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [96]:
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [97]:
# reflect an existing database into a new model
# reflect the tables
# We can view all of the classes that automap found
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()

['measurement', 'station']

In [98]:
# Save references to each table
measurement = Base.classes.measurement
station = Base.classes.station

In [99]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [129]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results
measurement_df = pd.read_sql_query('select * from measurement where date > "2020-09-09"', con=engine)

In [149]:
# Calculate the date 1 year ago from the last data point in the database
results = session.query(measurement.date).order_by((measurement.date).asc()).all()

dates = [result[0] for result in results]
min_date = dates[0]

'2010-01-01'

In [113]:
# Perform a query to retrieve the data and precipitation scores
measurement_df = pd.read_sql_query('select * from measurement', con=engine)

# Save the query results as a Pandas DataFrame and set the index to the date column
# Sort the dataframe by date
indexed_measurement_df = measurement_df.set_index('date').sort_index()
indexed_measurement_df.head()

# Use Pandas Plotting with Matplotlib to plot the data


,id,station,prcp,tobs
date,,,,
2010-01-01,1,USC00519397,0.08,65.0
2010-01-01,9008,USC00518838,0.21,72.0
2010-01-01,9519,USC00519523,0.15,75.0
2010-01-01,12188,USC00519281,0.15,70.0
2010-01-01,14960,USC00511918,0.05,66.0


In [122]:
# Use Pandas to calcualte the summary statistics for the precipitation data
indexed_measurement_df.describe()

,id,prcp,tobs
count,19550.000000,18103.000000,19550.000000
mean,9775.500000,0.160644,73.097954
std,5643.743217,0.468746,4.523527
min,1.000000,0.000000,53.000000
25%,4888.250000,0.000000,70.000000
50%,9775.500000,0.010000,73.000000
75%,14662.750000,0.110000,76.000000
max,19550.000000,11.530000,87.000000


In [69]:
# Design a query to show how many stations are available in this dataset?
session.query(measurement.station).distinct().count()

9

In [70]:
# What are the most active stations? (i.e. what stations have the most rows)?
# List the stations and the counts in descending order.
session.query(measurement.station, func.count(measurement.date)).\
    group_by(measurement.station).\
    order_by(func.count(measurement.date).desc()).all()

[('USC00519281', 2772),
 ('USC00519397', 2724),
 ('USC00513117', 2709),
 ('USC00519523', 2669),
 ('USC00516128', 2612),
 ('USC00514830', 2202),
 ('USC00511918', 1979),
 ('USC00517948', 1372),
 ('USC00518838', 511)]

In [156]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature of the most active station?
results = session.query(measurement.station, func.min(measurement.tobs), func.max(measurement.tobs), func.avg(measurement.tobs)).\
    group_by(measurement.station).\
    order_by(func.avg(measurement.tobs).desc()).all()
results

[('USC00514830', 58.0, 85.0, 74.8732970027248),
 ('USC00517948', 58.0, 87.0, 74.68440233236151),
 ('USC00519397', 56.0, 87.0, 74.55323054331865),
 ('USC00519523', 56.0, 85.0, 74.5436493068565),
 ('USC00518838', 58.0, 83.0, 72.72407045009784),
 ('USC00513117', 59.0, 85.0, 72.68918420081211),
 ('USC00519281', 54.0, 85.0, 71.66378066378067),
 ('USC00511918', 53.0, 87.0, 71.61596766043456),
 ('USC00516128', 58.0, 84.0, 70.91500765696784)]

In [157]:
# Choose the station with the highest number of temperature observations.
stations = [result[0] for result in results]
#most_frequent_station = stations[0]
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(measurement.station).filter(measurement.station == most_frequent_station).all()
results

[]